In [33]:
import pandas as pd
import requests
import time

In [ ]:
df_direcciones_destinos = pd.read_csv('../hospedajes_alvaro/direcciones_hoteles_de_destinos.csv')
df_hoteles_destinos = pd.read_csv('../hospedajes_alvaro/hospedajes_hoteles_de_destinos.csv')
df_propietarios_destinos = pd.read_csv('../hospedajes_alvaro/propietarios_hoteles_de_destinos.csv')

df_direcciones_alvaro = pd.read_csv('../hospedajes_alvaro/direcciones_merged_with_colab.csv')
df_hoteles_alvaro = pd.read_csv('../hospedajes_alvaro/hoteles_merged_with_colab.csv')
df_propietarios_alvaro = pd.read_csv('../hospedajes_alvaro/propietarios_merged_with_colab.csv')

df_direcciones_wada = pd.read_csv('../hospedajes_wada/output/direcciones.csv')
df_hoteles_wada = pd.read_csv('../hospedajes_wada/output/hoteles.csv')
df_propietarios_wada = pd.read_csv('../hospedajes_wada/output/propietarios.csv')

df_destinos = pd.read_csv('../destinos/destinos.csv')

In [7]:
df_hoteles_wada['destino_id'] = [0 for i in range(len(df_hoteles_wada))]

## Encontrar destino a partir de la direccion para hospedajes con destino_id=0

### Buscar destinos (tarda mucho)

In [86]:
def buscar_destino_por_coordenadas(lat, lon, espera=1):
    """
    Consulta la API de Nominatim con datos de latitud y longitud
    
    Parámetros:
        lat (float): Latitud
        lon (float): Longitud
        espera (int): Tiempo de espera entre solicitudes (segundos).
        
    Retorna:
        dict | None: Primer resultado de la búsqueda o None si no hay resultados.
    """
    url = "https://nominatim.openstreetmap.org/reverse?"
    params = {
        "lat": lat,
        "lon": lon,
        "zoom": 10,
        "format": "jsonv2",
        "addressdetails": 1,
        "extratags": 1,
    }

    headers = {
        'User-Agent': 'metabuscador-dtl',  # Si no se pone un user-agent se enoja la API
        'Accept': 'application/json',  # Para aceptar respuesta en JSON
    }

    try:
        response = requests.get(url, params=params, headers=headers)
        response.raise_for_status()
        resultados = response.json()

        if resultados:
            print(resultados)
            return resultados
        else:
            print('Sin resultados')
            return None
    except requests.RequestException as e:
        print(f"Error en la solicitud: {e}")
        return None
    finally:
        time.sleep(espera)  # Para evitar ser bloqueado por exceso de solicitudes

def encontrar_destinos(fila):
    resultado = buscar_destino_por_coordenadas(fila['lat'], fila['lon'])
    if resultado is not None:
        print(resultado)
        destino = dict()
        destino['nombre'] = resultado['name']
        destino['categoria'] = resultado['category']
        destino['tipo'] = resultado['type']
        destino['ciudad'] = resultado['address'].get('city', 'unkwown')
        destino['municipio'] = resultado['address'].get('county', 'unkwown')
        destino['estado'] = resultado['address'].get('state', 'unkwown')
        destino['codigopostal'] = resultado['address'].get('postcode', 'unkwown')
        destino['lat'] = resultado['lat']
        destino['lon'] = resultado['lon']
        bounding_box = resultado.get('boundingbox', (0,0,0,0))
        destino['min_lat'] = bounding_box[0]
        destino['max_lat'] = bounding_box[1]
        destino['min_lon'] = bounding_box[2]
        destino['max_lon'] = bounding_box[3]

        destinos_de_hospedajes_sin_destino.append(destino)


In [74]:
df_hoteles_sin_destino = df_hoteles[df_hoteles['destino_id'] == 0]
df_direcciones_hoteles_sin_destino = df_direcciones.loc[df_hoteles_sin_destino.index]


destinos_de_hospedajes_sin_destino = list()
df_direcciones_hoteles_sin_destino.apply(encontrar_destinos, axis=1)
# encontrar_destinos(df_direcciones_hoteles_sin_destino.iloc[0])

{'place_id': 289436496, 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. http://osm.org/copyright', 'osm_type': 'relation', 'osm_id': 5606660, 'lat': '20.7916423', 'lon': '-103.49110360410037', 'category': 'boundary', 'type': 'administrative', 'place_rank': 12, 'importance': 0.37942724125570976, 'addresstype': 'county', 'name': 'Zapopan', 'display_name': 'Zapopan, Región Centro, Jalisco, México', 'address': {'county': 'Zapopan', 'state_district': 'Región Centro', 'state': 'Jalisco', 'ISO3166-2-lvl4': 'MX-JAL', 'country': 'México', 'country_code': 'mx'}, 'extratags': {'wikidata': 'Q2143868', 'population': '1476491', 'INEGI:MUNID': '14120', 'population:date': '2020', 'source:population': 'https://www.inegi.org.mx/programas/ccpv/2020/#Datos_abiertos'}, 'boundingbox': ['20.5847883', '20.9982375', '-103.6650327', '-103.3059749']}
{'place_id': 289436496, 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. http://osm.org/copyright', 'osm_type': 'relation', 'osm_id': 5606660, 'lat

1355     None
1356     None
1357     None
1392     None
1408     None
         ... 
22804    None
22805    None
22806    None
22807    None
22808    None
Length: 2407, dtype: object

### Post procesado de los registros obtenidos

In [88]:
df_destinos_de_hospedajes_sin_destino = pd.DataFrame(destinos_de_hospedajes_sin_destino)
df_destinos_de_hospedajes_sin_destino.rename(columns={ 'mmax_lon': 'max_lon'}, inplace=True)

In [90]:
df_destinos_de_hospedajes_sin_destino.drop_duplicates(inplace=True)

## Merge

In [ ]:
df_direcciones = pd.concat([df_direcciones_destinos, df_direcciones_alvaro, df_direcciones_wada], ignore_index=True)
df_hoteles = pd.concat([df_hoteles_destinos, df_hoteles_alvaro, df_hoteles_wada], ignore_index=True)
df_propietarios = pd.concat([df_propietarios_destinos, df_propietarios_alvaro, df_propietarios_wada], ignore_index=True)

In [113]:
df_destinos_merged = pd.concat([df_destinos, df_destinos_de_hospedajes_sin_destino], ignore_index=True)
df_destinos_merged['lat'] = df_destinos_merged['lat'].astype(float)
df_destinos_merged['lon'] = df_destinos_merged['lon'].astype(float)
df_destinos_merged['min_lat'] = df_destinos_merged['min_lat'].astype(float)
df_destinos_merged['max_lat'] = df_destinos_merged['max_lat'].astype(float)
df_destinos_merged['min_lon'] = df_destinos_merged['min_lon'].astype(float)
df_destinos_merged['max_lon'] = df_destinos_merged['max_lon'].astype(float)
df_destinos_merged

,nombre,categoria,tipo,ciudad,municipio,estado,codigopostal,lat,lon,min_lat,max_lat,min_lon,max_lon,id_estado,id_municipio
0,Cabo San Lucas,place,city,Cabo San Lucas,Los Cabos,Baja California Sur,23470,22.893888,-109.920060,22.733888,23.053888,-110.080060,-109.760060,3.0,3008.0
1,Cancún,boundary,administrative,Cancún,Benito Juárez,Quintana Roo,unkwown,21.152747,-86.842576,20.970585,21.212375,-86.995617,-86.740526,23.0,23005.0
2,Loreto,place,town,unkwown,Loreto,Baja California Sur,23880,26.009791,-111.345252,25.969791,26.049791,-111.385252,-111.305252,3.0,3009.0
3,San José del Cabo,place,city,San José del Cabo,Los Cabos,Baja California Sur,23400,23.059836,-109.702515,22.899836,23.219836,-109.862515,-109.542515,3.0,3008.0
4,Ixtapa,boundary,administrative,unkwown,Ixtapa,Chiapas,unkwown,16.807623,-92.898650,16.684633,16.930706,-93.020620,-92.811072,7.0,7044.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
220,"Terrenos del Lote ""La Asunción""",boundary,administrative,unkwown,Tuxpan,Veracruz,92773,21.022504,-97.467442,20.906822,21.138042,-97.629830,-97.325482,NaN,NaN
221,"Terrenos del Lote ""La Isla""",boundary,administrative,unkwown,Tuxpan,Veracruz,unkwown,21.048225,-97.363323,20.963239,21.133552,-97.416243,-97.301713,NaN,NaN
222,Tuxpan,place,city,Tuxpan,Tuxpan,Veracruz,92800,20.951838,-97.404537,20.791838,21.111838,-97.564537,-97.244537,NaN,NaN
223,"Terrenos del Lote ""Álamo""",boundary,administrative,unkwown,Álamo Temapache,Veracruz,92730,20.867075,-97.676765,20.806173,20.927458,-97.701500,-97.662393,NaN,NaN


In [114]:
df_destinos_merged.dtypes

nombre           object
categoria        object
tipo             object
ciudad           object
municipio        object
estado           object
codigopostal     object
lat             float64
lon             float64
min_lat         float64
max_lat         float64
min_lon         float64
max_lon         float64
id_estado       float64
id_municipio    float64
dtype: object

## Agregar destino_id a hospedajes

In [96]:
def add_destino_id(fila):
    for index, row in df_destinos_merged.iterrows():
        if (fila.lat >= row.min_lat and fila.lat <= row.max_lat) and (fila.lon >= row.min_lon and fila.lon <= row.max_lon):
            return index+1
    return 0

In [115]:
# Como df_hoteles_destinos ya tiene asignado destino_id se omite.
df_hoteles.iloc[len(df_hoteles_destinos) : , df_hoteles.columns.get_loc('destino_id')] = df_direcciones.iloc[len(df_hoteles_destinos) : ].apply(add_destino_id, axis=1)

In [116]:
df_hoteles.destino_id.unique()

array([  1,   2,   3,   6,   7,   8,   9,  10,  11,  12,  13,  15,  16,
        17,  18,  19,  20,  21,  22,  23,  24,  25,  26,  27,  29,  30,
        31,  32,  33,  34,  35,  36,  37,  38,  39,  40,  41,  42,  43,
        44,  45,  47,  48,  50,  51,  52,  53,  54,  55,  56,  57,  58,
        59,  60,  61,  62,  63,  64,  65,  66,  67,  68,  69,  71,  72,
        73,  74,  75,  77,  78,  79,  80,  81,  82,  83,  84,  86,  87,
        88,  89,  90,  91,  92,  94,  95,  96,  97,  98,  99, 100, 101,
       102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 114, 116,
       117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129,
       131, 132, 133, 136, 137, 138, 140, 141, 142, 143, 144, 145,   4,
       146, 147, 148, 149, 150, 151, 156, 157,   5, 158, 159, 160, 161,
       162, 164, 165, 166,  46, 167, 168, 170,  14, 171, 174, 176, 177,
       178, 179, 180, 181, 183, 184, 185, 186, 187, 188, 190, 192, 193,
       194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 20

## Exportar

In [118]:
df_hoteles.to_csv('hoteles.csv', index=False)
df_direcciones.to_csv('direcciones.csv', index=False)
df_propietarios.to_csv('propietarios.csv', index=False)
df_destinos_merged.to_csv('destinos.csv', index=False)